## Active Learning with Regularization and Data Augmentation

After performing some unsupervised analysis, it's clear that there are a few classes that are hard to distinguish in the feature space. To help identify a strong separation boundary between these more fuzzy observations, we can apply two techniques:

1. Active Learning to query for examples which the model has a hard time discerning. 
2. Data Augmentation to create new samples from the points that the model has a hard time discerning. 

With active learning, we hope to be more thoughtful about which labels we choose to extract from the database since retrieving labels can be an expensive task. Moreover, with data augmentation, we hope to make our model more robust in classifying fuzzier data points without having to query the database for additional labels. Though augmented samples increase our model's training time (which likely has its own costs), they don't require us to pay any costs associated with generating labels. We are willing to make this tradeoff.

To leverage these methods in our approach, we build the following modeling pipeline:

1. Generate initial random sample of data and train on our CNN. We choose to collect an equal number of instances per class so the model can identify which classes are separable and which aren't. The remaining examples go into a sampling pool which our CNN can query.
2. Create new samples with data generator based on the initial random sample.
3. Train CNN on this initial sample and its generated examples.
4. Query for observations that model has most uncertainty in assigning a label.
5. Generate new data samples from queried data.
6. Train CNN on queried and new data samples. Remove the queried samples from the sampling pool.
7. Repeat steps 4-6.

We hope that our setup will allow us to build a classifier that hits our accuracy benchmark using as few labels as possible.

In [1]:
import keras
from keras.models import Sequential
from keras.layers import (
    Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Concatenate
)
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import Adam
from keras.layers.experimental.preprocessing import (
    RandomRotation, RandomFlip, RandomTranslation, RandomZoom
)
from modAL.models import ActiveLearner
from modAL.uncertainty import entropy_sampling, uncertainty_sampling, margin_sampling
from data.get_data import load_mnist
import numpy as np

In [2]:
# here y_train serves as our mock database to collect labels
X_train, y_train = load_mnist('data/f_mnist_data', kind='train')
X_test, y_test = load_mnist('data/f_mnist_data', kind='t10k')

# minmax scaling
X_train, X_test = X_train/255.0, X_test/255.0

In [3]:
X_train = X_train.reshape(60000, 28, 28, 1)
X_test = X_test.reshape(10000, 28, 28, 1)

y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

# assemble initial data sample for active learner
initial_idx = np.array([],dtype=int)
n_instances_per_class = 200
for i in range(10):
    idx = np.random.choice(np.where(y_train[:,i]==1)[0], size=n_instances_per_class, replace=False)
    initial_idx = np.concatenate((initial_idx, idx))

The next cell implements a simple data generator to create new training examples. To synthesize new examples, we apply some fairly standard image transformations to our input.  

In [4]:
from keras.layers import Concatenate

augmenter = Sequential([
    RandomFlip("horizontal_and_vertical"),
    RandomRotation(0.15),
    RandomTranslation(height_factor=(-.15, .15), width_factor=(-.15, .15)),
    RandomZoom(height_factor=(-.15, .15))
])

def generate_new_samples(X, y, augmenter=augmenter, num_samples=1):
    if num_samples > 1:
        sample_batch = Concatenate(axis=0)([augmenter(X) for _ in range(num_samples)])
        sample_labels = Concatenate(axis=0)([y for _ in range(num_samples)])
        return sample_batch.numpy(), sample_labels.numpy()
    else:
        sample_batch = augmenter(X).numpy()
        # sample_label is same as y
        return sample_batch, y

2021-12-27 12:22:08.786869: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
# forming the initial training data
X_initial = X_train[initial_idx]
y_initial = y_train[initial_idx]

# generate augmented samples
X_gen, y_gen = generate_new_samples(X_initial, y_initial, num_samples=1)

# generate the sampling pool
# remove the initial data from the training dataset
X_pool = np.delete(X_train, initial_idx, axis=0)
y_pool = np.delete(y_train, initial_idx, axis=0)

In [6]:
def create_keras_model():

    # model configs and hyperparams
    dim = (28,28,1)
    dropout_rate = .25
    
    model = Sequential([
        Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same', 
               input_shape=dim),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(dropout_rate),

        Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same'),
        BatchNormalization(),     
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(dropout_rate),
        
        Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'),
        BatchNormalization(),  
        MaxPooling2D(pool_size=(2, 2)),   
        Dropout(dropout_rate),
        
        Flatten(),
        
        Dense(1024, activation='relu'),
        Dropout(dropout_rate),
        
        Dense(512, activation='relu'),
        Dropout(dropout_rate),
        
        Dense(10, activation='softmax')
    ])

    adam = Adam(lr=0.001, decay=1e-6)
    model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

    return model

classifier = KerasClassifier(create_keras_model)

In [7]:
# training params
epochs = 40

# initialize ActiveLearner
learner = ActiveLearner(
    estimator=classifier,
    X_training=np.concatenate([X_initial, X_gen], axis=0), 
    y_training=np.concatenate([y_initial, y_gen], axis=0),
    query_strategy=margin_sampling,
    verbose=1,
    epochs=epochs,
)

# baseline score
print(f"baseline score: {learner.score(X_test, y_test)}")

2021-12-27 12:22:10.807925: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)


Epoch 1/40
125/125 [==============================] - 9s 70ms/step - loss: 2.4874 - accuracy: 0.3578
Epoch 2/40
125/125 [==============================] - 9s 70ms/step - loss: 1.2106 - accuracy: 0.5539
Epoch 3/40
125/125 [==============================] - 9s 69ms/step - loss: 1.0343 - accuracy: 0.6157
Epoch 4/40
125/125 [==============================] - 9s 70ms/step - loss: 0.9364 - accuracy: 0.6494
Epoch 5/40
125/125 [==============================] - 9s 71ms/step - loss: 0.8174 - accuracy: 0.6839
Epoch 6/40
125/125 [==============================] - 10s 79ms/step - loss: 0.7474 - accuracy: 0.7398
Epoch 7/40
125/125 [==============================] - 9s 68ms/step - loss: 0.7046 - accuracy: 0.7524
Epoch 8/40
125/125 [==============================] - 8s 68ms/step - loss: 0.6382 - accuracy: 0.7731
Epoch 9/40
125/125 [==============================] - 9s 69ms/step - loss: 0.6009 - accuracy: 0.7728
Epoch 10/40
125/125 [==============================] - 9s 70ms/step - loss: 0.5417 - accur

In [8]:
n_queries = 5
epochs=40
accuracy_arr = list()

# conduct several rounds of data querying and use the newly queried samples to improve model performance
for idx in range(n_queries):
    print(f'Query no. {(idx + 1)}')
    query_idx, query_instance = learner.query(X_pool, n_instances=400, verbose=0)
    
    # generate new samples
    X_gen, y_gen = generate_new_samples(X_pool[query_idx], y_pool[query_idx], num_samples=1)
    
    # train active learner on queried data and newly generated samples
    learner.teach(
        X=np.concatenate([X_pool[query_idx], X_gen], axis=0),
        y=np.concatenate([y_pool[query_idx], y_gen], axis=0), 
        verbose=1,
        epochs=epochs,
    )
    
    # remove queried instances from the sampling pool
    X_pool = np.delete(X_pool, query_idx, axis=0)
    y_pool = np.delete(y_pool, query_idx, axis=0)
    
    # evaluate performance
    model_accuracy = learner.score(X_test, y_test, verbose=0)
    accuracy_arr.append(model_accuracy)
    print(f"model accuracy: {model_accuracy}")

Query no. 1


/Users/imuzumdar/opt/miniconda3/envs/fashion_mnist/lib/python3.9/site-packages/tensorflow/python/keras/engine/sequential.py:425: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '


Epoch 1/40
150/150 [==============================] - 11s 69ms/step - loss: 2.5417 - accuracy: 0.3330
Epoch 2/40
150/150 [==============================] - 10s 69ms/step - loss: 1.2814 - accuracy: 0.5189
Epoch 3/40
150/150 [==============================] - 11s 70ms/step - loss: 1.1470 - accuracy: 0.5729
Epoch 4/40
150/150 [==============================] - 11s 71ms/step - loss: 1.0186 - accuracy: 0.6115
Epoch 5/40
150/150 [==============================] - 11s 72ms/step - loss: 0.9505 - accuracy: 0.6383
Epoch 6/40
150/150 [==============================] - 11s 72ms/step - loss: 0.8582 - accuracy: 0.6782
Epoch 7/40
150/150 [==============================] - 11s 72ms/step - loss: 0.8288 - accuracy: 0.6700
Epoch 8/40
150/150 [==============================] - 11s 71ms/step - loss: 0.7677 - accuracy: 0.6992
Epoch 9/40
150/150 [==============================] - 11s 72ms/step - loss: 0.7402 - accuracy: 0.7096
Epoch 10/40
150/150 [==============================] - 11s 72ms/step - loss: 0.688

/Users/imuzumdar/opt/miniconda3/envs/fashion_mnist/lib/python3.9/site-packages/tensorflow/python/keras/engine/sequential.py:425: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '


Epoch 1/40
175/175 [==============================] - 13s 69ms/step - loss: 2.6430 - accuracy: 0.3243
Epoch 2/40
175/175 [==============================] - 12s 70ms/step - loss: 1.3427 - accuracy: 0.4841
Epoch 3/40
175/175 [==============================] - 13s 71ms/step - loss: 1.1777 - accuracy: 0.5481
Epoch 4/40
175/175 [==============================] - 13s 72ms/step - loss: 1.0816 - accuracy: 0.5749
Epoch 5/40
175/175 [==============================] - 13s 73ms/step - loss: 1.0243 - accuracy: 0.6118
Epoch 6/40
175/175 [==============================] - 13s 73ms/step - loss: 0.9559 - accuracy: 0.6276
Epoch 7/40
175/175 [==============================] - 12s 71ms/step - loss: 0.9066 - accuracy: 0.6465
Epoch 8/40
175/175 [==============================] - 12s 70ms/step - loss: 0.8486 - accuracy: 0.6661
Epoch 9/40
175/175 [==============================] - 13s 76ms/step - loss: 0.7862 - accuracy: 0.6887
Epoch 10/40
175/175 [==============================] - 13s 76ms/step - loss: 0.762

...

Epoch 1/40
200/200 [==============================] - 14s 69ms/step - loss: 2.4219 - accuracy: 0.3153
Epoch 2/40
200/200 [==============================] - 14s 69ms/step - loss: 1.3256 - accuracy: 0.4869
Epoch 3/40
200/200 [==============================] - 14s 72ms/step - loss: 1.2058 - accuracy: 0.5270
Epoch 4/40
200/200 [==============================] - 15s 73ms/step - loss: 1.1017 - accuracy: 0.5692
Epoch 5/40
200/200 [==============================] - 15s 73ms/step - loss: 1.0445 - accuracy: 0.5882
Epoch 6/40
200/200 [==============================] - 14s 72ms/step - loss: 0.9840 - accuracy: 0.6121
Epoch 7/40
200/200 [==============================] - 14s 72ms/step - loss: 0.9215 - accuracy: 0.6330
Epoch 8/40
200/200 [==============================] - 15s 73ms/step - loss: 0.8767 - accuracy: 0.6468
Epoch 9/40
200/200 [==============================] - 14s 72ms/step - loss: 0.8324 - accuracy: 0.6702
Epoch 10/40
200/200 [==============================] - 15s 73ms/step - loss: 0.783

/Users/imuzumdar/opt/miniconda3/envs/fashion_mnist/lib/python3.9/site-packages/tensorflow/python/keras/engine/sequential.py:425: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '


Epoch 1/40
225/225 [==============================] - 15s 66ms/step - loss: 2.3939 - accuracy: 0.3073
Epoch 2/40
225/225 [==============================] - 15s 66ms/step - loss: 1.3686 - accuracy: 0.4589
Epoch 3/40
225/225 [==============================] - 15s 65ms/step - loss: 1.2439 - accuracy: 0.5055
Epoch 4/40
225/225 [==============================] - 15s 66ms/step - loss: 1.1691 - accuracy: 0.5434
Epoch 5/40
225/225 [==============================] - 15s 66ms/step - loss: 1.0597 - accuracy: 0.5686
Epoch 6/40
225/225 [==============================] - 15s 66ms/step - loss: 1.0008 - accuracy: 0.5970
Epoch 7/40
225/225 [==============================] - 15s 65ms/step - loss: 0.9551 - accuracy: 0.6147
Epoch 8/40
225/225 [==============================] - 15s 67ms/step - loss: 0.9270 - accuracy: 0.6268
Epoch 9/40
225/225 [==============================] - 121s 539ms/step - loss: 0.8866 - accuracy: 0.6322
Epoch 10/40
225/225 [==============================] - 45s 202ms/step - loss: 0.

/Users/imuzumdar/opt/miniconda3/envs/fashion_mnist/lib/python3.9/site-packages/tensorflow/python/keras/engine/sequential.py:425: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '


Epoch 1/40
250/250 [==============================] - 35s 137ms/step - loss: 2.4411 - accuracy: 0.3212
Epoch 2/40
250/250 [==============================] - 47s 189ms/step - loss: 1.3732 - accuracy: 0.4534
Epoch 3/40
250/250 [==============================] - 93s 375ms/step - loss: 1.2085 - accuracy: 0.5145
Epoch 4/40
250/250 [==============================] - 25s 101ms/step - loss: 1.1675 - accuracy: 0.5281
Epoch 5/40
250/250 [==============================] - 43s 171ms/step - loss: 1.0515 - accuracy: 0.5797
Epoch 6/40
250/250 [==============================] - 466s 2s/step - loss: 1.0183 - accuracy: 0.5919
Epoch 7/40
250/250 [==============================] - 531s 2s/step - loss: 0.9854 - accuracy: 0.5994
Epoch 8/40
250/250 [==============================] - 241s 969ms/step - loss: 0.9220 - accuracy: 0.6315
Epoch 9/40
250/250 [==============================] - 17s 70ms/step - loss: 0.8767 - accuracy: 0.6505
Epoch 10/40
250/250 [==============================] - 17s 69ms/step - loss: 

In [9]:
len(learner.X_training)

8000

So far, we've shown our model 8000 total samples (half of which are synthetic and have zero labeling costs), and our model's accuracy is ~88.1%. We are already close to our baseline results and we've used 60% less labels! 

Can we improve our model's performance by querying more samples?

In [10]:
# 3 more times
for idx in range(n_queries, n_queries + 3):
    print(f'Query no. {(idx + 1)}')
    query_idx, query_instance = learner.query(X_pool, n_instances=400, verbose=0)
    X_gen, y_gen = generate_new_samples(X_pool[query_idx], y_pool[query_idx], num_samples=1)
    learner.teach(
        X=np.concatenate([X_pool[query_idx], X_gen], axis=0),
        y=np.concatenate([y_pool[query_idx], y_gen], axis=0), 
        verbose=1,
        epochs=epochs,
    )
    
    # remove queried instance from pool
    X_pool = np.delete(X_pool, query_idx, axis=0)
    y_pool = np.delete(y_pool, query_idx, axis=0)
    
    # evaluate performance
    model_accuracy = learner.score(X_test, y_test, verbose=0)
    accuracy_arr.append(model_accuracy)
    print(f"model accuracy: {model_accuracy}")

Query no. 6


/Users/imuzumdar/opt/miniconda3/envs/fashion_mnist/lib/python3.9/site-packages/tensorflow/python/keras/engine/sequential.py:425: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '


Epoch 1/40
275/275 [==============================] - 18s 65ms/step - loss: 2.3780 - accuracy: 0.3158
Epoch 2/40
275/275 [==============================] - 18s 65ms/step - loss: 1.3652 - accuracy: 0.4513
Epoch 3/40
275/275 [==============================] - 20s 73ms/step - loss: 1.3034 - accuracy: 0.4780
Epoch 4/40
275/275 [==============================] - 20s 74ms/step - loss: 1.1778 - accuracy: 0.5192
Epoch 5/40
275/275 [==============================] - 20s 71ms/step - loss: 1.1317 - accuracy: 0.5401
Epoch 6/40
275/275 [==============================] - 19s 71ms/step - loss: 1.0520 - accuracy: 0.5712
Epoch 7/40
275/275 [==============================] - 20s 74ms/step - loss: 0.9956 - accuracy: 0.58780s - loss: 0.9954 - accura
Epoch 8/40
275/275 [==============================] - 20s 73ms/step - loss: 0.9362 - accuracy: 0.6249
Epoch 9/40
275/275 [==============================] - 20s 72ms/step - loss: 0.9023 - accuracy: 0.6341
Epoch 10/40
275/275 [==============================] - 1

/Users/imuzumdar/opt/miniconda3/envs/fashion_mnist/lib/python3.9/site-packages/tensorflow/python/keras/engine/sequential.py:425: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '


Epoch 1/40
300/300 [==============================] - 21s 68ms/step - loss: 2.2742 - accuracy: 0.3046
Epoch 2/40
300/300 [==============================] - 21s 69ms/step - loss: 1.3854 - accuracy: 0.4483
Epoch 3/40
300/300 [==============================] - 20s 67ms/step - loss: 1.2618 - accuracy: 0.4815
Epoch 4/40
300/300 [==============================] - 20s 68ms/step - loss: 1.1708 - accuracy: 0.5172
Epoch 5/40
300/300 [==============================] - 21s 69ms/step - loss: 1.1168 - accuracy: 0.5473
Epoch 6/40
300/300 [==============================] - 21s 68ms/step - loss: 1.0778 - accuracy: 0.5541
Epoch 7/40
300/300 [==============================] - 21s 71ms/step - loss: 1.0053 - accuracy: 0.5929
Epoch 8/40
300/300 [==============================] - 22s 73ms/step - loss: 0.9856 - accuracy: 0.6028
Epoch 9/40
300/300 [==============================] - 21s 69ms/step - loss: 0.9099 - accuracy: 0.6271
Epoch 10/40
300/300 [==============================] - 20s 68ms/step - loss: 0.885

/Users/imuzumdar/opt/miniconda3/envs/fashion_mnist/lib/python3.9/site-packages/tensorflow/python/keras/engine/sequential.py:425: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '


Epoch 1/40
325/325 [==============================] - 22s 66ms/step - loss: 2.2181 - accuracy: 0.3111
Epoch 2/40
325/325 [==============================] - 22s 69ms/step - loss: 1.3659 - accuracy: 0.4443
Epoch 3/40
325/325 [==============================] - 23s 70ms/step - loss: 1.2623 - accuracy: 0.4830
Epoch 4/40
325/325 [==============================] - 25s 77ms/step - loss: 1.1726 - accuracy: 0.5156
Epoch 5/40
325/325 [==============================] - 23s 71ms/step - loss: 1.1153 - accuracy: 0.5427
Epoch 6/40
325/325 [==============================] - 23s 70ms/step - loss: 1.0412 - accuracy: 0.5756
Epoch 7/40
325/325 [==============================] - 23s 70ms/step - loss: 1.0417 - accuracy: 0.5751
Epoch 8/40
325/325 [==============================] - 22s 67ms/step - loss: 0.9756 - accuracy: 0.5901
Epoch 9/40
325/325 [==============================] - 22s 68ms/step - loss: 0.9234 - accuracy: 0.6221
Epoch 10/40
325/325 [==============================] - 22s 68ms/step - loss: 0.911

At this point, we've shown our model 5,200 real samples, or 8.667% of the train dataset, and we've surpassed the performance of our baseline!

We also have that the test accuracy has increased on almost every iteration of querying (the notable exception being the last iteration). So, for the msot part, our queries are helping the model classify examples of unseen data!

In [11]:
# one more iteration *fingers crossed*
query_idx, query_instance = learner.query(X_pool, n_instances=400, verbose=0)
X_gen, y_gen = generate_new_samples(X_pool[query_idx], y_pool[query_idx], num_samples=1)
learner.teach(
    X=np.concatenate([X_pool[query_idx], X_gen], axis=0),
    y=np.concatenate([y_pool[query_idx], y_gen], axis=0), 
#         X=X_pool[query_idx],
#         y=y_pool[query_idx],
    verbose=1,
    epochs=epochs,
)

# remove queried instance from pool
X_pool = np.delete(X_pool, query_idx, axis=0)
y_pool = np.delete(y_pool, query_idx, axis=0)

# evaluate performance
model_accuracy = learner.score(X_test, y_test, verbose=0)
accuracy_arr.append(model_accuracy)
print(f"model accuracy: {model_accuracy}")

/Users/imuzumdar/opt/miniconda3/envs/fashion_mnist/lib/python3.9/site-packages/tensorflow/python/keras/engine/sequential.py:425: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '


Epoch 1/40
350/350 [==============================] - 25s 70ms/step - loss: 2.2005 - accuracy: 0.3122
Epoch 2/40
350/350 [==============================] - 24s 70ms/step - loss: 1.3550 - accuracy: 0.4488
Epoch 3/40
350/350 [==============================] - 24s 70ms/step - loss: 1.2721 - accuracy: 0.4840
Epoch 4/40
350/350 [==============================] - 25s 71ms/step - loss: 1.1984 - accuracy: 0.5134
Epoch 5/40
350/350 [==============================] - 25s 71ms/step - loss: 1.1004 - accuracy: 0.5412
Epoch 6/40
350/350 [==============================] - 25s 71ms/step - loss: 1.0681 - accuracy: 0.5465
Epoch 7/40
350/350 [==============================] - 25s 71ms/step - loss: 1.0190 - accuracy: 0.5733
Epoch 8/40
350/350 [==============================] - 25s 71ms/step - loss: 0.9869 - accuracy: 0.5825
Epoch 9/40
350/350 [==============================] - 25s 71ms/step - loss: 0.9516 - accuracy: 0.5982
Epoch 10/40
350/350 [==============================] - 25s 71ms/step - loss: 0.903

We hit 90% accuracy! Let's see how much data we used in total.

In [12]:
print(f"Number of total samples trained on Active Learner: {len(learner.X_training)}")

Number of total samples trained on Active Learner: 11200


In [13]:
print(f"Max Accuracy on Test Set: {max(accuracy_arr)}")

Max Accuracy on Test Set: 0.9006999731063843


In [15]:
max_idx = accuracy_arr.index(max(accuracy_arr))
num_samples_max_accuracy = n_instances_per_class * 10 + (max_idx + 1) * 400

print(f"Number of total samples trained at point of Maximum Accuracy: {num_samples_max_accuracy * 2}")
print(f"Number of augmented samples trained at point of Maximum Accuracy: {num_samples_max_accuracy}")
print(f"Number of real samples trained at point of Maximum Accuracy: {num_samples_max_accuracy}")
print(f"Percentage of train data used at point of Maximum Accuracy: {num_samples_max_accuracy/X_train.shape[0]}")

Number of total samples trained at point of Maximum Accuracy: 11200
Number of augmented samples trained at point of Maximum Accuracy: 5600
Number of real samples trained at point of Maximum Accuracy: 5600
Percentage of train data used at point of Maximum Accuracy: 0.09333333333333334


Using active learning and data augmentation, we were able to hit our target accuracy (barely, but still) using only 5,600 queried observations from the labels database. In the end, we only used 9.33% of our train data, which is around half the data we used in our baseline experiment. And, with some finer-grained model tuning, I think it's possible we could hit our accuracy target with even fewer observations. But given the time constraints of this problem, that is outside the scope of this analysis. 